## Clase 5

La Regresion Logistica es un modelo simple utilizado para realizar clasificacion binaria. Su mayor limitante esta en que **no** es capaz de clasificar datos que no son linealmente separable.

![img](xor.png)

Existe una variacion de la Regresion Logistica para clasificar multiclases, donde el modelo es el siguiente:

$$
p(y=c|x, \theta) = \frac{exp(\theta^T_c x)}{\sum ^C_{v=1} exp(\theta^T_v x)}
$$

donde C es el numero de clases. Esto se conoce comunmente como un **clasificador de maxima entropia**.
En scikit learn, entrenar un regresor logistico binario o multiclase es transparente para el usuario.


En las siguientes clases veremos como solucionar este problema utilizando tecnicas de agrupamiento de modelos. Pero antes, veremos 2 modelos no-parametricos utilizados en clasificacion multiclase.

## K-Nearest Neighboor (KNN)

K Vecinos mas Cercanos es un modelo no parametrico comunmente usado para la clasificacion multiclase. Tambien puede ser usado para solucionar problemas de regresion.
Dependiendo del problema, la salida del KNN sera:

- En clasificacion es la clase, decidida a partir del voto de sus vecinos.
- En regresion es un valor numero representando el promedio de la caracteristica correspondiente de los vecinos.

Este modelo se caracteriza por estar dictado por los datos, donde los calculos se llevan acabo al momento de la prediccion. Ademas, al estar basado en distancias, las caracteristicas con distintas escalas (como unidades de medida) afectan fuertemente su rendimiento, requiriendo una normalizacion previa de los datos.

#### Funcionamiento del modelo

La fase de entrenamiento del modelo solo consiste en almacenar los datos. Esto se puede hacer bajo algun criterio como simplemente un vector o un arbol binario, para facilitar la busqueda luego.

La fase de clasificacion consiste en tener un vector que quiero clasificar $x_i$ y encontrar los $K$ vecinos mas cercanos, es decir, que tengan la minima distancia total. Comunmente se utiliza la distancia euclideana para valores reales y la distancia hamming para discretos. Luego en base a la clase de los vecinos, asignarle una clase a $x_i$ bajo algun criterio, como mayoria. Tambien se puede ponderar la votacion de cada vecino con el inverso de su distancia, asi los vecinos mas lejanos importan menos en la decision.

![](knn.gif)

En Scikit Learn, KNN utiliza la misma logica del resto de modelos:

[sklearn.neighbors.KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

`KNeighborsClassifier.fit()` para ajustar el modelo (guardar los datos) \
`KNeighborsClassifier.predict()` para clasificar observaciones

**Ventajas**

- Rapido con pequenos y medianos datasets
- Algoritmo simple, ademas el resultado es interpretable.
- No se hacen supuestos de los datos, pues deja que los mismo decidan sobre nueva entrada. Util en escenarios no lineales.
- Es simple y rapido agregarle mas informacion al modelo (una nueva observacion)

**Desventajas**

- La precision depende de la calidad de los datos. Esto significa que es sensible al ruido.
- Con datasets grandes, el tiempo de prediccion puede ser alto.
- Sensible a la escala de los datos y las caracteristicas irrelevantes.


## Arbol de Decision

El Arbol de Decision (Decision Tree) es otro modelo no-parametrico capaz de hacer clasificacion y regresion. Se considera parte de los modelos con mejor interpretabilidad y sencillez.

### ¿Qué se aprende y cómo?

Para entender más precisamente cómo y qué <b>aprenden</b> los algoritmos de tipo Decision Tree, implantamos a continuación un algoritmo desde cero en Python. En camino, presentaremos los conceptos de <b>Gini impurity</b> e <b>Information Gain</b>.

### 1. Algoritmos de tipo Decision Tree: CART

Existen varias variaciones de algoritmos Decision Tree, entre las cuales: ID3, C4.5, C5.0, CART <a href="https://en.wikipedia.org/wiki/Decision_tree">(ver wikipedia)</a>
Cada algoritmo comparta la misma idea: a cada nivel del árbol de decisión, el modelo formula una pregunta permitiendo poco a poco de llegar a una predicción. ¿Cómo el algoritmo sabe qué preguntas hacer y en qué orden? Es lo que vamos a responder a través del ejemplo de CART. 

CART (Classification and Regression Trees) inicializa la raíz del árbol con el dataset de entranemiento completo (ver Figura). Luego busca dividir los datos del dataset con una pregunta. Los nodos siguientes reciben solamente los datos que corresponde a la respuesta. Se reitera este proceso hasta desenredar totalmente los datos (cada hoja del árbol debería tener idealmente un solo tipo de label).

<img src="./img/CART.png"></img>

En nuestro ejemplo, el nodo 1 está totalmente desenredado (label: "Grape"). El nodo 2 tiene dos labeles, entonces preguntamos otra pregunta:

<img src="./img/CART2.png"></img>

Para construir un árbol eficiente, el punto importante es identificar qué preguntas formular y cuándo. Por lo tanto, necesitamos <b>cuantificar</b> en qué medida una pregunta permite desenredar los labeles. Para hacer eso se utiliza 2 métricas:
- el coeficiente de '<b>Gini impurity</b>': mide que tan desenredados están los labeles de un nodo.
- el coeficiente de '<b>Information Gain</b>': mide cuánto una pregunta permite bajar el 'Gini impurity'.

<img src="./img/CART3.png"></img>

Utilizaremos estas métricas para estimar qué preguntas hacer.

### 2. ¿Qué preguntas formular?

<img src="./img/CART4.png"></img>

Para saber qué preguntas formular, cada nodo itera sobre las características de los datos a su disposición y define una lista de preguntas posibles.

### 3. Partición del dataset

Una vez una pregunta elegida, se divide los datos en dos según la respuesta a la pregunta.

<img src="./img/CART5.png"></img>

### 4. Coeficiente de Gini impurity

El coeficiente de Gini impurity representa la probabilidad de ser incorrecto si asigna aleatoriamente una etiqueta a un ejemplo del mismo conjunto. Este se calcula por nodo, como:

$$
\sum_{i=1}^J p(x)\cdot p(1-x)
$$

Con J siendo la cantidad de elementos en la particion, y $p(x)$ la probabilidad de etiquetar incorrectamente un ejemplo nuevo.

### 5. Information Gain

La métrica de Information Gain permite medir qué pregunta optimiza el coeficiente de Gini impurity.

Por cada nodo, empezamos por medir el coeficiente de Gini impurity de los labeles disponibles. Luego, por cada pregunta calculamos el coeficiente de Gini impurity de los dos sub-conjuntos de datos obtenidos.

<img src="./img/CART-8.png"></img>

Calculamos la incerteza (<i>impurity</i>) promedia ponderada para los dos subconjuntos de datos obtenidos. Por ejemplo:

<img src="./img/CART-9.png"></img>

Finalmente, conservamos la pregunta que permite optimizar la ganancia de información (Information Gain). En nuestro ejemplo:

<b>Information Gain = 0.64 - 0.5 = 0.14</b>

Cuando se detiene el algoritmo del árbol de decisión?
- Cuando todos los nodos son puros
- Cuando se alcanza la profundidad máxima indicada por el usuario

Respecto a la profundidad del árbol
- Mientras más profundo es el árbol mayor es su complejidad
- También se hace más propenso a sobre-ajustar el ruido en los datos

Cómo evitar el sobreajuste? \
Poda (pruning): Eliminar ramas causadas por el ruido y peculiaridades 
del set de entrenamiento
- Poda funciona de acuerdo al error de validación
- Pre-poda: La rama no crece si incrementa el error de validación
- Post-poda: La rama será removida si hacerlo disminuye el error de validación


**Ventajas**
- Sin parámetros sensibles
- No hace supuesto sobre la distribución de los features
- No necesita escalar las features (pre-processing)
- Selección de características como parte del algoritmo
- Maneja datos numéricos y categóricos
- Robusto a outliers

**Desventajas**
- Fronteras de decisión perpendiculares a los ejes de características (hiper-rectángulo)
- Inestable: Una pequeña cantidad de ruido produce un árbol muy distinto. 
- Tendencia al sobreajuste. Puede mitigarse usando poda.



En Scikit Learn usar un arbol sigue el mismo esquema que el resto de modelos:

[sklearn.tree.DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

`DecisionTreeClassifier.fit()` para ajustar el modelo (guardar los datos) \
`DecisionTreeClassifier.predict()` para clasificar observaciones

Existen herramientas para exportar el arbol construido en una representacion grafica

Para esto es necesario instalar las librerias `pydot` y `graphviz`
```python
from sklearn.externals.six import StringIO
import pydot

clf = DcisionTreeClassifier()
dot_data = StringIO()

features=['Altura', 'Peso', 'Edad']
classes=['0', '1']
tree.export_graphviz(clf,out_file=dot_data,feature_names=features,class_names=classes, filled=True, 
                     rounded=True, impurity=False)

graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph[0].write_png('modelo.png')
```

![](modelo.png)